In [1]:
from TMfunctions import *
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
# Cross Validation
from sklearn.model_selection import KFold

C:\Users\david\AppData\Local\Temp\ipykernel_20212\803022442.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB

from gensim.models import Word2Vec
import numpy as np

# Pretrained

In [3]:
# This model can be downloaded from https://bio.nlplab.org/
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('Pretrained Models/PubMed-and-PMC-w2v.bin', binary=True)

In [4]:
train = pd.read_csv('Data/StopWords/train.csv', index_col = 0)
test = pd.read_csv('Data/StopWords/test.csv', index_col = 0)
data = pd.concat([train,test])

In [5]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [model[word] for word in words if word in model]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [19]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1(build_results(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB = 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5}
result

fold 1:
{'DT': 0.537, 'RF': 0.612, 'NB': 0.653}
 
fold 2:
{'DT': 0.546, 'RF': 0.608, 'NB': 0.658}
 
fold 3:
{'DT': 0.544, 'RF': 0.629, 'NB': 0.669}
 
fold 4:
{'DT': 0.542, 'RF': 0.622, 'NB': 0.668}
 
fold 5:
{'DT': 0.538, 'RF': 0.61, 'NB': 0.662}
 


{'DT': 0.5414000000000001, 'RF': 0.6162, 'NB': 0.662}

In [6]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.loc[train_index]
    test = data.loc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['medical_abstract']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['medical_abstract']]))
    
    Kresult = micro_f1SVM(build_resultsSVM(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB, SVM = 0, 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
        SVM += result['SVM']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5, 'SVM': SVM/5}
result

fold 1:
{'DT': 0.535, 'RF': 0.612, 'NB': 0.653, 'SVM': 0.707}
 
fold 2:
{'DT': 0.545, 'RF': 0.62, 'NB': 0.658, 'SVM': 0.715}
 
fold 3:
{'DT': 0.55, 'RF': 0.632, 'NB': 0.669, 'SVM': 0.722}
 
fold 4:
{'DT': 0.547, 'RF': 0.621, 'NB': 0.668, 'SVM': 0.709}
 
fold 5:
{'DT': 0.54, 'RF': 0.598, 'NB': 0.662, 'SVM': 0.704}
 


{'DT': 0.5434, 'RF': 0.6165999999999999, 'NB': 0.662, 'SVM': 0.7114}